# **Geometric Parametrization**
This lab focuses on performing a ROM on a parametric system with variable domain geometry.

In [ ]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

In [ ]:
!mkdir -p release
%cd release 
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

In [ ]:
import numpy as np
import GeDiM4Py as gedim
from scipy.sparse.linalg import splu
import time

In [ ]:
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## Poisson problem on variable geometry

Solve the following equation on the domain ${\tilde{\Omega}} = \tilde{\Omega}_1 \cup \tilde{\Omega}_2 =  (0, 1) \times (0, 1) \cup (1, 1+
\mu) \times (0, 1)$

$$
\begin{cases}
\nabla \cdot (\nabla u) = f & \text{in } \tilde{Ω}\\
u = 0 & \text{in } \partial \tilde{Ω}
\end{cases}
$$

The parametric space is $\mathcal P = [1, 3.5]$.

### Map to reference domain

**Goal**: build the ROM space using a reference domain

We choose as **reference domain** $\Omega$ the case $\mu = 1$.

Thus, $\Omega = \Omega_1 \cup \Omega_2 = [0,1] \times [0,1] \cup [1,2] \times [0,1]$.

The affine transformations are the following:
$$\tilde{\mathbf{x}} = \Phi_{\Omega_1}(\mathbf{x}, \mu) = \mathbb{I}\mathbf{x} + \mathbf{0}= \begin{bmatrix}
1 & 0 \\
0 & 1 
\end{bmatrix}\mathbf{x} + \begin{pmatrix}
0\\
0
\end{pmatrix} \quad \forall \tilde{\mathbf{x}} \in \tilde{\Omega_1}$$
$$\tilde{\mathbf{x}} = \Phi_{\Omega_2}(\mathbf{x}, \mu) = \mathbb{A}\mathbf{x} + \mathbf{c} = \begin{bmatrix}
\mu & 0 \\
0 & 1 
\end{bmatrix}\mathbf{x} + \begin{pmatrix}
1-\mu\\
0
\end{pmatrix} \quad \forall \tilde{\mathbf{x}} \in \tilde{\Omega_2}$$

Notice that
$$J_{\Phi_{\Omega_2}} =\begin{bmatrix}
\mu & 0 \\
0 & 1 
\end{bmatrix} ⇒ J^{-1}_{\Phi_{\Omega_2}} =\begin{bmatrix}
\frac{1}{\mu} & 0 \\
0 & 1 
\end{bmatrix}$$
and $|J_{\Phi_{\Omega_2}}| = \mu$

In [ ]:
def Map(points, mu):
  numPoints = points.shape[1]
  mappedPoints = np.copy(points)

  for p in range(1, numPoints):
    if (points[0, p] > 1.0 + 1.0e-8):
      mappedPoints[0, p] = mu * points[0, p] + (1. - mu)
  return mappedPoints

### Problem data

For this Lab we would like to find the solution
$$u = 16 x y (1 + \mu - x) (1-y)$$

Thus, the forcing term reads
$$f = 32 [x(1+\mu-x) + y(1-y)]$$

In [ ]:
def forcing_term(points):
  return 32.0 * (points[1,:] * (1.0 - points[1,:]) + points[0,:] * (1.0 + MU_TILDE - points[0,:]))
def exact_solution(points):
  return 16.0 * (points[1,:] * (1.0 - points[1,:]) * points[0,:] * (1.0 + MU_TILDE - points[0,:]))
def exact_derivative_solution(direction, points):
  if direction == 0:
    values = 16.0 * (1.0 + MU_TILDE - 2.0 * points[0,:]) * points[1,:] * (1.0 - points[1,:])
  elif direction == 1:
    values = 16.0 * (1.0 - 2.0 * points[1,:]) * points[0,:] * (1.0 + MU_TILDE - points[0,:])
  else:
    values = np.zeros(points.shape[1])
  return values

### Suggestions

The problem on the reference domain $Ω$ shall be computing applying the transformation function to the original problem.

Thus we will have:

$$\tilde{a}(\tilde{u}, \tilde{v}) = \int_{\tilde{\Omega}} \tilde{\nabla} \tilde{u}(\tilde{\mathbf{x}})\cdot \tilde{\nabla} \tilde{v}(\tilde{\mathbf{x}}) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} \tilde{\nabla} \tilde{u}(\Phi(\mathbf{x})) \cdot \tilde{\nabla} \tilde{v}(\Phi(\mathbf{x}))|J_{\Phi}| ⇒ \dots$$
and:
$$\tilde{f}(\tilde{v}) = \int_{\tilde{\Omega}} \tilde{f}(\tilde{\mathbf{x}})\tilde{v}(\tilde{\mathbf{x}}) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} \tilde{f}(\Phi(\mathbf{x})) \tilde{v}(\Phi(\mathbf{x}))|J_{\Phi}| ⇒ \dots$$

Moreover, you will need a new function to assemble the problem on the reference domain, able to assemble the stiffness matrix with a symmetric tensor $\mathbb{K}$:
$$\mathbb{K}= \begin{bmatrix}
k_1 & k_2 \\
k_2 & k_3 
\end{bmatrix}$$

In [ ]:
def K(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  k = np.zeros((3, numPoints), order='F') # this is the k tensor => for each point p [k_1, k_2, k_3]^T
  for p in range(0, numPoints):
      values[0, p] = k_1
      values[1, p] = k_2
      values[2, p] = k_3
  return k.ctypes.data

gedim.AssembleAnisotropicStiffnessMatrix(K, problemData, lib)

## Other Useful Functions

In [ ]:
def normX(v, X):
  return np.sqrt(np.transpose(v) @ X @ v)

def ProjectSystem(AQH, fQH, B):
    AQN = []
    fQN = []
    for AH in AQH:
        AQN.append(np.copy(np.transpose(B) @ AH @ B))
    for fH in fQH:
        fQN.append(np.copy(np.transpose(B) @ fH))
    return [AQN, fQN]

def Solve_full_order(AQH, fQH, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQH[0]
    f = thetaF_mu[0] * fQH[0]
    for i in range(1, len(AQH)):
        A += thetaA_mu[i] * AQH[i]
    for i in range(1, len(fQH)):
        f += thetaF_mu[i] * fQH[i]
    return gedim.LUSolver(A, f, lib)

def Solve_reduced_order(AQN, fQN, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQN[0]
    f = thetaF_mu[0] * fQN[0]
    for i in range(1, len(AQN)):
        A += thetaA_mu[i] * AQN[i]
    for i in range(1, len(fQN)):
        f += thetaF_mu[i] * fQN[i]
    return np.linalg.solve(A, f)

def POD(AQH, fQH, X, N_max, tol):
    #### snapshot matrix creation
    snapshot_matrix = []

    for mu in training_set:
        snapshot = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))
        snapshot_matrix.append(np.copy(snapshot))
    
    snapshot_matrix = np.array(snapshot_matrix) 

    ### covariance matrix
    C = snapshot_matrix @ X @ np.transpose(snapshot_matrix) ## metti inner product
    L_e, VM_e = np.linalg.eig(C)
    eigenvalues = []
    eigenvectors = []

    for i in range(len(L_e)):
        eig_real = L_e[i].real
        eig_complex = L_e[i].imag
        assert np.isclose(eig_complex, 0.)
        eigenvalues.append(eig_real)
        eigenvectors.append(VM_e[i].real)

    total_energy = sum(eigenvalues)
    retained_energy_vector = np.cumsum(eigenvalues)
    relative_retained_energy = retained_energy_vector/total_energy

    if all(flag==False for flag in relative_retained_energy >= (1.0 - tol)):
        N = N_max
    else:
        N = np.argmax(relative_retained_energy >= (1.0 - tol)) + 1

    # Create the basis function matrix
    basis_functions = []
    for n in range(N):
        eigenvector =  eigenvectors[n]
        # basis = (1/np.sqrt(M))*np.transpose(snapshot_matrix)@eigenvector 
        basis = np.transpose(snapshot_matrix) @ eigenvector
        norm = normX(basis, X)
        # norm = np.sqrt(np.transpose(basis)@basis)
        basis /= norm
        basis_functions.append(np.copy(basis))

    return [N, np.transpose(np.array(basis_functions))]

def TestSingleParameter(AQH, fQH, AQN, fQN, B, mu):
    reduced_solution = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
    full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

    ###### plot #######
    proj_reduced_solution = B @ reduced_solution

    ### computing error
    error_function = full_solution - proj_reduced_solution
    error_norm_squared_component = np.transpose(error_function) @ X @ error_function
    abs_err_ROM = np.sqrt(abs(error_norm_squared_component))

    full_solution_norm_squared_component = np.transpose(full_solution) @ X @ full_solution
    rel_err_ROM = abs_err_ROM / np.sqrt(abs(full_solution_norm_squared_component))

    solutionStrong = np.zeros(problemData['NumberStrongs'])

    map_mu = mu[0]
    mappedMesh = Map(mesh, map_mu)
    mappedDofs = Map(dofs, map_mu)
    mappedStrongs = Map(strongs, map_mu)

    gedim.PlotSolution(mappedMesh, mappedDofs, mappedStrongs, proj_reduced_solution, solutionStrong, "MOR Solution")
    gedim.PlotSolution(mappedMesh, mappedDofs, mappedStrongs, full_solution, solutionStrong, "FOR Solution")
      
    return [rel_err_ROM, abs_err_ROM]